In [18]:
import requests
from bs4 import BeautifulSoup
page = requests.get('https://www.jdsports.co.uk/men/mens-clothing/jackets/brand/nike/')
html = page.text
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<!-- MESH ver. 2022-wk02 -->
<html lang="en">
<head>
<!-- charset and compatibility -->
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="IE=EmulateIE7; IE=EmulateIE9" http-equiv="X-UA-Compatible">
<!-- device settings -->
<meta content="" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<!-- site information -->
<title>Men's Nike Jackets | Windrunner, Flex Pro, Club | JD Sports </title>
<meta content="Men's Nike Jackets | Windrunner, Flex Pro, Club | JD Sports " name="title"/>
<meta content="Shop our range of Men - Nike Jackets online at JD Sports ✓Free Standard Delivery Over £70  ✓10% Student Discount ✓Buy Now, Pay Later" name="description">
<meta content="" name="keywords">
<!-- canonicals & alternates -->
<link href="https://www.jdsports.co.uk/men/mens-clothing/jackets/brand/nike/" rel="canonical"/>
<link href="https://m.jdsports.co.uk/men/mens-clothing/jacket

In [27]:
# soup.find('form', {'label': 'sort by'})
soup.find('label').text

'Sort by'

In [1]:
# import selenium
# from selenium.webdriver import Chrome
# driver = Chrome('./chromedriver')
# driver.get('https://www.jdsports.co.uk/men/mens-clothing/jackets/brand/nike/')

/var/folders/g2/zwqkmzwj2vs3pv8rydkch4m40000gn/T/ipykernel_20043/1400593293.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome('./chromedriver')


In [2]:
# from webdriver_manager.chrome import ChromeDriverManager

In [29]:
# driver = Chrome(ChromeDriverManager().install())
# driver.get('https://www.jdsports.co.uk/men/mens-clothing/jackets/brand/nike/')



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/FKhan/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/var/folders/g2/zwqkmzwj2vs3pv8rydkch4m40000gn/T/ipykernel_20043/2798737221.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome(ChromeDriverManager().install())


In [19]:
# import selenium
# from selenium import webdriver
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from selenium.common.exceptions import TimeoutException
# import time

In [4]:
from scraper import Scraper 
bot = Scraper()
bot.accept_cookies()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/FKhan/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


In [5]:
bot.keys_search('nike caps')

In [ ]:
bot.next_button()

<selenium.webdriver.remote.webelement.WebElement (session="1ce55851c5d7850482175ee1b9b0d67d", element="73ace4c0-c480-43b8-b65a-555fbd6d2c28")>

In [6]:
container = bot.product_container()

In [7]:
from lib2to3.pgen2 import driver
from time import time
from typing_extensions import Self
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager


In [8]:
list_properties = container.find_elements(By.XPATH, './ul/li')

In [9]:
link_list = []
for property in list_properties:
    link_list.append(property.find_element(By.TAG_NAME, 'a').get_attribute('href'))

In [10]:
import time
import uuid
property_dict = {'link':[],
                 'price':[],
                 'name':[],
                 'id' :[],
                 'UUID':[],
                }

for link in link_list[0:5]:
    bot.driver.get(link)
    time.sleep(1)
    property_dict['link'].append(link)
    price = bot.driver.find_element(By.XPATH, '//span[@class="pri"]')
    property_dict['price'].append(price.text)
    name = bot.driver.find_element(By.XPATH, '//h1[@itemprop="name"]')
    property_dict['name'].append(name.text)
    id = bot.driver.find_element(By.XPATH, '//span[@class="product-code"]')
    property_dict['id'].append(id.text)
    property_dict['UUID'].append(uuid.uuid4())
    

In [11]:
property_dict

{'link': ['https://www.jdsports.co.uk/product/grey-nike-heritage-86-swoosh-cap/16474497/',
  'https://www.jdsports.co.uk/product/green-nike-sportswear-heritage86-cap/16453825/',
  'https://www.jdsports.co.uk/product/orange-nike-heritage-86-swoosh-cap/16450304/',
  'https://www.jdsports.co.uk/product/black-nike-sportswear-classic-99-trucker-cap/16253171/',
  'https://www.jdsports.co.uk/product/black-nike-heritage86-cap-womens/16154660/'],
 'price': ['£15.00', '£25.00', '£15.00', '£25.00', '£23.00'],
 'name': ["Nike Heritage '86 Swoosh Cap",
  'Nike Sportswear Heritage86 Cap',
  "Nike Heritage '86 Swoosh Cap",
  'Nike Sportswear Classic 99 Trucker Cap',
  "Nike Heritage86 Cap Women's"],
 'id': ['Product Code: 16474497/474027',
  'Product Code: 16453825/474048',
  'Product Code: 16450304/474030',
  'Product Code: 16253171/506339',
  'Product Code: 16154660/506521'],
 'UUID': [UUID('bf63a330-9a14-4f51-94b3-3403aaa550d6'),
  UUID('e962ddcb-5fc7-4170-9f39-00fdec55991f'),
  UUID('492bd370-ec6

In [41]:
import json
from uuid import UUID


class UUIDEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, UUID):
            return obj.hex
        return json.JSONEncoder.default(self, obj)
j = json.dumps(property_dict, cls=UUIDEncoder, indent=4)
f = open('data.json', 'w')
print(j, file=f)
f.close()

In [104]:
import pandas as pd 
pd.DataFrame(property_dict)

,link,price,name,id,UUID
0,https://www.jdsports.co.uk/product/green-nike-...,£25.00,Nike Sportswear Heritage86 Cap,Product Code: 16453825/474048,47c4eab2-732d-4a1a-ab74-5f5bee1b9175
1,https://www.jdsports.co.uk/product/orange-nike...,£15.00,Nike Heritage '86 Swoosh Cap,Product Code: 16450304/474030,a5e1fd74-af76-49e6-b943-0f4537690fd5
2,https://www.jdsports.co.uk/product/black-nike-...,£25.00,Nike Sportswear Classic 99 Trucker Cap,Product Code: 16253171/506339,135ca3ce-6383-4d0b-8ada-189d82231ca6
3,https://www.jdsports.co.uk/product/white-nike-...,£23.00,Nike Heritage86 Cap Women's,Product Code: 16253133/506525,151dde99-6c74-4161-81aa-70b7f0832910
4,https://www.jdsports.co.uk/product/pink-nike-f...,£18.00,Nike Featherlight Running Cap,Product Code: 16232870/473947,69e18dcc-f73e-4b93-b67e-2137f813b5ee


In [110]:
import urllib
import urllib.request
x = urllib.request.urlopen('https://www.jdsports.co.uk')
print(x.read())

b'<!DOCTYPE html>\n<!-- MESH ver. 2022-wk02 -->\n<html lang="en">\n\t<head>\n\t\t<!-- charset and compatibility -->\n<meta charset="UTF-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7; IE=EmulateIE9" />\n\n<!-- device settings -->\n<meta name="viewport" content="">\n<meta name="format-detection" content="telephone=no">\n\n<!-- site information -->\n<title>JD Sports | Shop Online Now </title>\n<meta name="title" content="JD Sports | Shop Online Now ">\n    <meta name="description" content="JD Sports is the leading trainer & sports fashion retailer in the UK \xe2\x9c\x93Free Standard Delivery Over \xc2\xa370 \xe2\x9c\x9310% Student Discount \xe2\x9c\x93Buy Now, Pay Later" />\n<meta name="keywords" content="jd sports, clothing, dresses, tops, jackets, jumpers, blouses, shirts" />\n\n<!-- canonicals & alternates -->\n<link rel="canonical" href="https://www.jdsports.co.uk/" />\n<link rel="alternate" media="only s